In [17]:
import re
import math
import sys

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

import pythainlp

from sklearn.cluster import DBSCAN

from data_tokenizer import load_corpus

### Load Data

In [3]:
file_name = 'ผู้บริโภค - TrueCoffee.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 353
1 clusters


### Preprocess Corpus

#### Remove Words

In [6]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 953 words
filter frequent words: 374 words
filter letter words: 372 words
filter stop words: 221 words


In [14]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [15]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)
average_doc_size

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### DBSCAN

In [16]:
def get_bow(corpus):
    new_dict = Dictionary(corpus)

    # new_dict.filter_extremes(no_below=2, no_above=1, keep_n=len(new_dict))
    # print(len(new_dict))

    unique_words = [new_dict[id] for id in range(len(new_dict))]
    array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)
    
    for i, doc in enumerate(corpus):
        for word in doc:
            array[i, new_dict.token2id[word]] += 1

        ## normalization
        if len(doc) != 0:
            array[i] = numpy.divide(array[i], len(doc))

    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [29]:
def predict_cluster(bow_corpus):
    model = DBSCAN().fit(bow_corpus)
    predicted_labels = model.labels_
    predicted_labels += 1

    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    
    return result

In [30]:
bow_corpus = get_bow(filtered_corpus)
result = predict_cluster(bow_corpus)
result.head()

,comment,tokenized_comment,label,predicted_label
0,ร้านสวยมาก สวยจนอยากบอกว่า เอาเงินแต่งร้าน ส่ง...,"[ร้าน, สวย, สวย, เงิน, พนง, เรียน, ชง, กาแฟ]",,1
1,ไม่ควรส่งโปรมาให้เรา ถ้าสิทธิ์มันเต็มแล้วจากใจ...,"[โปร, สิทธิ์, เต็ม, ลูกค้า, ทรูมูฟ]",,1
2,ราคาสตาบัค รสชาติออลคาเฟ่,"[ราคา, สตาบัค, รสชาติ]",,1
3,ช่วงนี้เนตช้ามากเลยครับ ช่วยปรับปรุงด้วยครับ ร...,"[เนต, ปรับปรุง, เดือน, แพง, คุ้ม]",,1
4,ไม่ ชอบ การ แบ่ง ชนชั้น ในร้านกาแฟ ผู้ถือบัตรอ...,"[ชอบ, ร้าน, กาแฟ, บัตร, นั่ง, บัตร, นั่ง, จ่าย...",,1


### Result

In [31]:
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
num_clusters = len(label_count[0])
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4]), array([119, 211,   7,  10,   6]))
		percent
0  |	119	1.0


In [32]:
comment_widget = widgets.ToggleButtons(
    options=[num + 1 for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_comment_widget_click(change):
    clear_output()
    display(comment_widget)
    for index, value in result[result['predicted_label'] == change['new']]['comment'].iteritems():
        print(index, value)

comment_widget.observe(on_comment_widget_click, names='index')
on_comment_widget_click({'new' : 0})

ToggleButtons(index=4, options=(1, 2, 3, 4, 5), value=5)

221 อร่อยแล้วไมาต้องปร้บ
247 ไม่เน้นน้ำเน้นเบเกอรี่ เบเกอรี่อร่อย
322 ไม่อร่อยค่ะ
323 ไม่อร่อยเลย
324 ไม่อร่อย
346 ไม่อร่อย


In [33]:
token_widget = widgets.ToggleButtons(
    options=[num + 1 for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_token_widget_click(change):
    clear_output()
    display(token_widget)
    for index, value in result[result['predicted_label'] == change['new']]['tokenized_comment'].iteritems():
        print(index, value)

token_widget.observe(on_token_widget_click, names='index')
on_token_widget_click({'new' : 0})

ToggleButtons(index=1, options=(1, 2, 3, 4, 5), value=2)

0 ['ร้าน', 'สวย', 'สวย', 'เงิน', 'พนง', 'เรียน', 'ชง', 'กาแฟ']
1 ['โปร', 'สิทธิ์', 'เต็ม', 'ลูกค้า', 'ทรูมูฟ']
2 ['ราคา', 'สตาบัค', 'รสชาติ']
3 ['เนต', 'ปรับปรุง', 'เดือน', 'แพง', 'คุ้ม']
4 ['ชอบ', 'ร้าน', 'กาแฟ', 'บัตร', 'นั่ง', 'บัตร', 'นั่ง', 'จ่าย', 'ราคา', 'กาแฟ', 'ไหม']
5 ['เน็ต', 'บ้าน', 'หลุด', 'หาย', 'เน็ต', 'ปรับปรุง']
6 ['ลูกค้า', 'สั่ง', 'รบกวน', 'พนักงาน', 'สิทธิ์', 'เต็ม', 'กก']
7 ['โปร', 'ดืม', 'เต็ม', 'ทำ', 'กิน', 'สตาบัค', 'ดี', 'ไม']
8 ['ไทย', 'น้ำ', 'แข็ง', 'แน่นแก้ว', 'ขนาด', 'กิน', 'แป๊บ', 'แก้ว', 'น้ำ', 'แข็ง', 'แก้ว']
9 ['อร่อย', 'ดี', 'อย่า', 'ผม']
10 ['ผม', 'ผม', 'ชอบ', 'รสชาติ', 'กาแฟ', 'แก้ว', 'แบรนด์', 'แพง', 'กะ', 'คน', 'ไทย', 'ลด', 'ลูกค้า', 'ทรู', 'สาขา', 'คน', 'คิว', 'เหมือน', 'แบรนด์', 'นางเงือก']
11 ['เต็ม', 'พนักงาน', 'กด', 'ตอน', 'คืน']
12 ['กาแฟ', 'จืด', 'เปล่า', 'ชอบ', 'กิน', 'แพง', 'โปร', 'รู้', 'โปร', 'งง']
13 ['ชาเย็น', 'อร่อย', 'ชอบ', 'น้ำ', 'แข็ง', 'ปั่น', 'น้ำ', 'ไหม', 'ดู', 'สอง', 'ราคา']
14 ['เนต', 'ดี', 'ดู', 'ไหร่', 'เดือน', 'หาย']
15 ['ท